# Reader Monad

## Outline

* Incentive for the Reader monad

* Definition of the Reader monad

* Reader monad examples

In this lesson, we will learn about the Reader monad type and how you can use it.

## Incentive for Reader monad

Imagine we have a global state which is represented by a variable that holds multiple data (e.g. a record syntax variable). 

If we have several functions that need to use part of that data and they call each other, we need to pass that variable from one to another.

Below is an example where we read 4 variables from the operating system environment and store them into a variable of type `Environment`. 

We process this data in 4 function calls and the last function writes the processed data to a configuration file.

In [ ]:
import Data.Maybe (fromMaybe)
import System.Environment (lookupEnv)

data Environment = Environment
    { param1 :: String
    , param2 :: String
    , param3 :: String
    , param4 :: String
    }

loadEnv :: IO Environment
loadEnv = do
  p1 <- lookupEnv "PATH"
  p2 <- lookupEnv "SHELL"
  p3 <- lookupEnv "HOSTNAME"
  p4 <- lookupEnv "HOME"
  return $ Environment
    (fromMaybe "" p1)
    (fromMaybe "" p2)
    (fromMaybe "" p3)
    (fromMaybe "" p4)

func1 :: Environment -> IO ()
func1 env = do
    let content = "System path includes: \n" ++ param1 env ++ "\n\n"
    func2 env content

func2 :: Environment -> String -> IO ()
func2 env content = do
    let updatedContent = content ++ "Shell used is: \n" ++ param2 env ++ "\n\n"
    func3 env updatedContent

func3 :: Environment -> String -> IO ()
func3 env content = do
    let updatedContent = content ++ "Hostname is: \n" ++ param3 env ++ "\n\n"
    func4 env updatedContent

func4 :: Environment -> String -> IO ()
func4 env content = do
    let updatedContent = content ++ "Current home directory is: \n" ++ param4 env ++ "\n\n"
        filePath = param4 env ++ "/conf.txt"
    writeFile filePath updatedContent

We can now write the main execution of our program where we read in the environment and process the data with our 4 functions.

In [ ]:
main1 :: IO ()
main1 = do
  env <- loadEnv
  print "Writing configuration to file: conf.txt"
  func1 env

main1

This is a simple example, but as complexity of programs increases you have to pass around a lot of data. 

For this reason the **Reader monad** was created which can store the state in its environment variable. 

Then it can pass this state variable around between different Reader monads that decide which part of the data to use.

## Definition of the Reader monad

The definition of the **Reader monad** is following: 
```haskell
type Reader r = ReaderT r Identity
newtype ReaderT r m a 
```
Constructors for **ReaderT**:
```haskell
runReaderT :: r -> m a
```

Another definition shown below uses multi-parameter type classes and funDeps, which are not standard Haskell 2010. 
```haskell
newtype Reader e a = Reader { runReader :: (e -> a) }

instance Monad (Reader e) where
    return a         = Reader $ \e -> a
    (Reader r) >>= f = Reader $ \e -> runReader (f (r e)) e
```

To make use of the Reader monad we help ourself with the functions `ask` and `runReader`.

The `ask` function can retrieve the environment variable. The `runReader` function can retrieve the final value of the Reader monad.

They are both included in the module `Control.Monad.Reader`. The `ask` function is defined in the `MonadReader` type class.

```haskell
class Monad m => MonadReader r m | m -> r where
    {-# MINIMAL (ask | reader), local #-}
    -- | Retrieves the monad environment.
    ask   :: m r
    ask = reader id

    -- | Executes a computation in a modified environment.
    local :: (r -> r) -- ^ The function to modify the environment.
          -> m a      -- ^ @Reader@ to run in the modified environment.
          -> m a

    -- | Retrieves a function of the current environment.
    reader :: (r -> a) -- ^ The selector function to apply to the environment.
           -> m a
    reader f = do
      r <- ask
      return (f r)

-- | Retrieves a function of the current environment.
asks :: MonadReader r m
    => (r -> a) -- ^ The selector function to apply to the environment.
    -> m a
asks = reader
```

Below is a simple example how to define a reader monad and use the `ask` and `runReader` functions.

In [ ]:
import Control.Monad.Reader

age :: Reader Int String
age = do
    env <- ask
    return ("My age is: " ++ (show env))

ageDouble :: Reader Int String
ageDouble = do
    env <- ask
    return ("Two times my age is: " ++ (show $ 2*env))

bothAges :: Reader Int String
bothAges = do
    a <- age
    ad <- ageDouble
    return (a ++ "\n" ++ ad)

main2 :: IO ()
main2 = do
    myAge <- read <$> getLine
    print $ (runReader bothAges) myAge

main2

We see that when calling the `age` and `ageDouble` Reader monads the environment of the `bothAges` Reader monad is passed onto them and they used this data. 

## Reader monad examples

Let's implement the example from the first chapter with the use of the **Reader monad**.

In [ ]:
processEnv :: Reader Environment String
processEnv = do
    conf1 <- func1'
    conf2 <- func2'
    conf3 <- func3'
    conf4 <- func4'
    let conf = conf1 ++ "\n\n" ++
               conf2 ++ "\n\n" ++
               conf3 ++ "\n\n" ++
               conf4 ++ "\n\n"
    return conf
  
func1' :: Reader Environment String
func1' = do
    env <- ask
    let content = "System path includes: \n" ++ param1 env
    return content

func2' :: Reader Environment String
func2' = do
    env <- ask
    let content = "Shell used is: \n" ++ param2 env
    return content

func3' :: Reader Environment String
func3' = do
    env <- ask
    let content = "Hostname is: \n" ++ param3 env
    return content

func4' :: Reader Environment String
func4' = do
    env <- ask
    let content = "Current home directory is: \n" ++ param4 env
    return content

main3 :: IO ()
main3 = do
    env <- loadEnv
    let configuration = runReader processEnv env
        filePath = param4 env ++ "/conf.txt"
    print "Writing configuration to file: conf.txt"
    writeFile filePath configuration

main3

We see that the functions `func1'` to `func4'` get more cleaner as the functions in the first example. 

The `env` variable is passed from the `processEnv` function to all the other Reader monad functions.

Below is another more example on how to lookup variables if you have them stored in a list (taken from HaskellWiki and adapted). 

The list contains tuples where the first part is the variable name and the second is variable either of type `String` or of type `Template`. 

What we want to show here is that it is also possible to pass in data to your Reader monad beside the intial environment.

In [ ]:
data Template = S String | V Template | T Template deriving Show

data Environment = Env {templates::[(String,Template)],
                        variables::[(String,String)]}

-- lookup a variable from the environment
lookupVar :: String -> Environment -> Maybe String
lookupVar name env = lookup name (variables env)

-- lookup a template from the environment
lookupTemplate :: String -> Environment -> Maybe Template
lookupTemplate name env = lookup name (templates env)

-- resolve a template into a string
resolve :: Template -> Reader Environment (String)
resolve (S s)    = return s
resolve (V t)    = do varName  <- resolve t
                      varValue <- asks (lookupVar varName)
                      return $ maybe "" id varValue
resolve (T t)    = do tmplName <- resolve t
                      body     <- asks (lookupTemplate tmplName)
                      return $ maybe "" show body

template1 :: Template
template1 = V (S "varName1")

template2 :: Template
template2 = T (S "varName2")

myEnv :: Environment
myEnv = Env {
      templates = [("varName1",template1),("varName2",template2)],
      variables = [("varName1","value1"),("varName2","value2")]
    }

main :: IO ()
main = do
    print $ runReader (resolve template1) myEnv
    print $ runReader (resolve template2) myEnv

main

## Recap

In this lesson we've discussed:

- the motivation for introducing the Reader monad type 

- the definition of the Reader monad type and a simple example

- examples that use the Reader monad type to solve a problem